# A Gentle Introduction to Text Summarization in Machine Learning

---

## Two Major Types of Text Summarization:
    - Extraction-based summarization
    - Abstraction-based summarization

## Steps to Perform Text Summarization:
    1. Convert the paragraph into sentences.
    2. Perform text processing.
    3. Perform tokenization.
    4. Evaluated the weighted occurrence frequency of the words. 
    5. Substitute words with their weighted frequencies.

In [8]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize

import bs4 as BeautifulSoup
from urllib import urlopen